In [5]:
from IPython import get_ipython
ip = get_ipython()
ip.run_line_magic("reload_ext", "autoreload")
ip.run_line_magic("autoreload", "2")

import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
import ipyutils as util
np.set_printoptions(suppress=True)
sp.init_printing(use_latex=True)

# Machine numbers

In disem modul wird allgemein in Base 10 mit vier Nachkommastellen gearbeitet.

In [11]:
from machine_accuracy import convert
from machine_accuracy import accuracy

num_float = 58.126654
num_scientific = {
    "mantissa": 0.58126654,
    "exponent": 2,
    "base": 10
}

limits = {
    "mantissa_length": 4,
    "exp_min": 0,
    "exp_max": 3,
    "base": 10
}

x = 5.3451
x_err = 5.3454

float_info = {
    "digits": 8,
    "base": 10
}

def func(x):
    return x ** 2 + 2 * x

def derivative(x):
    return 2 * x + 2

util.dis(f'{num_float} as scientific', convert.to_scientific(num_float))
util.dis(f'{num_scientific} as float', convert.to_float(**num_scientific))
print("")
util.dis(f'{limits} range', accuracy.limits(**limits))
util.dis(f'Absolute error of x', accuracy.absolute_error(approximation=x_err, exact=x))
util.dis(f'Relative error of x', accuracy.relative_error(approximation=x_err, exact=x))
util.dis(f'EPS for number with mantissa length {float_info["digits"]}', accuracy.eps(**float_info))
util.dis(f'Condition of func', accuracy.condition(func=func, derivative=derivative, x=x))
util.dis('Lower condition is better!')
util.dis(expr=accuracy.condition(func=func, derivative=derivative, x=x))

58.126654 as scientific: $\left( 0.58126654, \  10, \  2\right)$

{'mantissa': 0.58126654, 'exponent': 2, 'base': 10} as float: $58.126654$

{'mantissa_length': 4, 'exp_min': 0, 'exp_max': 3, 'base': 10} range: $\left( 0.1, \  999.9\right)$

Absolute error of x: $0.000299999999999301$

Relative error of x: $5.6126171633702 \cdot 10^{-5}$

EPS for number with mantissa length 8: $5.0 \cdot 10^{-8}$

Condition of func: $1.72770962954895$

Lower condition is better!

# Fixed-point iteration

The equation must first be transformed into its fixed-point form:

$$f(x) = 0 \rarr F(x) = x$$

In [17]:
import numpy as np

def start(A: np.ndarray, max_depth):
    return step(A, np.eye(A.shape[0]), 0, max_depth)

def step(A, P, depth, max_depth):
    Q, R = np.linalg.qr(A)
    if depth > max_depth:
        return R@Q, P@Q
    else:
        return step(R@Q, P@Q, depth + 1, max_depth)


#A = np.array([1,-2,0,2,0,1,0,-2,1]).reshape(3,3)

A = np.array([6,1,2,1,2,1,5,0,2,-1,2,0,5,-1,0,1,2,-1,6,1,2,-1,0,1,7]).reshape(5,5)
A_i, P = start(A, 200)
print(A)
print()
print(A_i)
print()
print(P)
print()
print(np.linalg.eig(A)[0])
print(np.linalg.eig(A)[1])

[[ 6  1  2  1  2]
 [ 1  5  0  2 -1]
 [ 2  0  5 -1  0]
 [ 1  2 -1  6  1]
 [ 2 -1  0  1  7]]

[[ 9.46410162 -0.          0.         -0.          0.        ]
 [ 0.          7.58836399 -0.         -0.          0.        ]
 [-0.         -0.          6.59357935 -0.         -0.        ]
 [ 0.          0.         -0.          2.81805666 -0.        ]
 [ 0.         -0.         -0.         -0.          2.53589838]]

[[ 0.63053937 -0.1342784   0.38589667 -0.22459826 -0.62050689]
 [ 0.19052975  0.57578471  0.33454367 -0.54376736  0.47388604]
 [ 0.19052975 -0.34756137  0.61495696  0.49006068  0.47388604]
 [ 0.41053456  0.63105853 -0.20818937  0.62008812 -0.07331043]
 [ 0.60106431 -0.36250174 -0.56360396 -0.17089159  0.40057561]]

[2.53589838 2.81805666 9.46410162 6.59357935 7.58836399]
[[-0.62050689  0.22459826  0.63053937 -0.38589667  0.1342784 ]
 [ 0.47388604  0.54376736  0.19052975 -0.33454367 -0.57578471]
 [ 0.47388604 -0.49006068  0.19052975 -0.61495696  0.34756137]
 [-0.07331043 -0.62008812  0

In [27]:
import numpy as np

steps = 0

def error_below_threshold(old_x: np.ndarray, new_x: np.ndarray, threshold):
    return np.linalg.norm(new_x - old_x, 2) < threshold

def vectoriteration(A, start, threshold=10**-4):
    v = (A @ start) / np.linalg.norm(A @ start, 2)
    global steps
    steps += 1
    if error_below_threshold(start, v, threshold):
        return v, (v.T @ A @ v) / (v.T @ v)
    else:
        return vectoriteration(A, v, threshold)

A = np.array([1,1,0,3,-1,2,2,-1,3]).reshape(3,3)
start = np.array([1,0,0]).reshape(3,1)
print(A)
print()
print(start)
print()
steps = 0
v, l = vectoriteration(A, start)
print(v)
print()
print(l)
print()
print(f'Done in {steps} steps')

[[ 1  1  0]
 [ 3 -1  2]
 [ 2 -1  3]]

[[1]
 [0]
 [0]]

[[0.29816784]
 [0.59627983]
 [0.74534979]]

[[3.00007493]]

Done in 14 steps
